In [1]:
import numpy as np
from docplex.mp.advmodel import AdvModel
from qiskit_optimization.translators import from_docplex_mp

mdl = AdvModel(name="Portfolio optimization")

In [2]:
N = 5   # Number of assets 
M = 3   # Number of levels

lambda_risk = 5
lambda_linking = 10
lambda_budget = 20
lambda_diversification = 15
lambda_cardinality = 10

R = 100   # Risk tolerance parameter
B = 50    # Total budget
D = 4    # Diversification requirement
K = 1     # Cardinality constraint


# Vector of mu_i values, expected returns for each asset 
mu = np.random.uniform(low=0.1, high=1.0, size=N)

# sigma_ij values, representing the covariance between assets i and j
sigma = np.random.uniform(low=0.01, high=0.1, size=(N, N))
# c_i values, representing the cost of each asset
c = np.random.uniform(low=1, high=10, size=N)
line_seperator = "----------------------------------------"
print(line_seperator)
print("N(Number of assets): ", N)
print("M(Number of levels): ", M)
print(line_seperator)
print("lambda_risk(Risk penalty coefficient): ", lambda_risk)
print("lambda_linking(Linking penalty coefficient): ", lambda_linking)
print("lambda_budget(Budget penalty coefficient): ", lambda_budget)
print("lambda_diversification(Diversification penalty coefficient): ", lambda_diversification)
print("lambda_cardinality(Cardinality penalty coefficient): ", lambda_cardinality)
print(line_seperator)
print("R(Risk tolerance parameter): ", R)
print("B(Total budget): ", B)
print("D(Diversification requirement): ", D)
print("K(Cardinality constraint): ", K)
print(line_seperator)
print("mu(Expected returns for each asset): ", mu)
print("sigma(Covariance between assets): ", sigma)
print("c(Cost of each asset): ", c)




----------------------------------------
N(Number of assets):  5
M(Number of levels):  3
----------------------------------------
lambda_risk(Risk penalty coefficient):  5
lambda_linking(Linking penalty coefficient):  10
lambda_budget(Budget penalty coefficient):  20
lambda_diversification(Diversification penalty coefficient):  15
lambda_cardinality(Cardinality penalty coefficient):  10
----------------------------------------
R(Risk tolerance parameter):  100
B(Total budget):  50
D(Diversification requirement):  4
K(Cardinality constraint):  1
----------------------------------------
mu(Expected returns for each asset):  [0.44091755 0.40151632 0.63356516 0.25443645 0.65186984]
sigma(Covariance between assets):  [[0.02177301 0.07631229 0.05589295 0.01777496 0.09086639]
 [0.09444558 0.01843316 0.04159563 0.05671592 0.08006804]
 [0.06196048 0.09639793 0.01169722 0.07819212 0.04987505]
 [0.01040307 0.01040763 0.03096709 0.07259146 0.03995639]
 [0.06765425 0.07020271 0.04528523 0.09068628 

In [3]:
z = {(i, l): mdl.binary_var(name=f"z_{i}_{l}") for i in range(N) for l in range(M)}
y = [mdl.binary_var(name=f"y_{i}") for i in range(N)]

#objective function
obj = mdl.sum(mu[i] * mdl.sum(2**l * z[i, l] for l in range(M)) for i in range(N))

penalty_risk = mdl.sum(mdl.sum(2**l * z[i, l] for l in range(M)) * sigma[i][j] * mdl.sum(2**l * z[j, l] for l in range(M)) for i in range(N) for j in range(N)) - R
# risk_term = lambda_risk * penalty_risk ** 2  
risk_term = lambda_risk * penalty_risk 

penalty_budget = mdl.sum(c[i] * mdl.sum(2**l * z[i, l] for l in range(M)) for i in range(N)) - B
# budget_term = lambda_budget * penalty_budget ** 2
budget_term = lambda_budget * penalty_budget

penalty_diversification = mdl.sum(mdl.sum(2**l * z[i, l] for l in range(M)) for i in range(N)) - D
# diversification_term = lambda_diversification * penalty_diversification ** 2
diversification_term = lambda_diversification * penalty_diversification 

penalty_cardinality = mdl.sum(y[i] for i in range(N)) - K
# cardinality_term = lambda_cardinality * penalty_cardinality ** 2
cardinality_term = lambda_cardinality * penalty_cardinality

penalty_linking = mdl.sum(mdl.sum(2**l * z[i, l] for l in range(M)) for i in range(N)) - D * mdl.sum(y[i] for i in range(N))
# linking_term = lambda_linking * penalty_linking ** 2
linking_term = lambda_linking * penalty_linking 

mdl.minimize(risk_term + budget_term+ diversification_term + cardinality_term + linking_term - obj)
qp = from_docplex_mp(mdl)
print(qp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Portfolio optimization

Minimize
 obj: 109.495126682021 z_0_0 + 218.990253364043 z_0_1 + 437.980506728085 z_0_2
      + 224.430714302980 z_1_0 + 448.861428605959 z_1_1 + 897.722857211918 z_1_2
      + 63.491573707028 z_2_0 + 126.983147414056 z_2_1 + 253.966294828111 z_2_2
      + 72.111778684964 z_3_0 + 144.223557369928 z_3_1 + 288.447114739857 z_3_2
      + 155.894905292939 z_4_0 + 311.789810585878 z_4_1 + 623.579621171756 z_4_2
      - 30 y_0 - 30 y_1 - 30 y_2 - 30 y_3 - 30 y_4 + [ 0.217730102372 z_0_0^2
      + 0.870920409489 z_0_0*z_0_1 + 1.741840818978 z_0_0*z_0_2
      + 1.707578683227 z_0_0*z_1_0 + 3.415157366453 z_0_0*z_1_1
      + 6.830314732906 z_0_0*z_1_2 + 1.178534301798 z_0_0*z_2_0
      + 2.357068603597 z_0_0*z_2_1 + 4.714137207194 z_0_0*z_2_2
      + 0.281780324614 z_0_0*z_3_0 + 0.563560649228 z_0_0*z_3_1
      + 1.127121298457 z_0_0*z_3_2 + 1.585206382978 z_0_0*z_4_0
      + 3.170412765957 z_

In [4]:
# General imports
import numpy as np
import warnings

warnings.filterwarnings("ignore")

# Pre-defined ansatz circuit, operator class and visualization tools
from qiskit.circuit.library import QAOAAnsatz
from qiskit.quantum_info import SparsePauliOp
from qiskit.visualization import plot_distribution

# Qiskit Runtime
from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_ibm_runtime import SamplerV2 as Sampler

# SciPy minimizer routine
from scipy.optimize import minimize

# rustworkx graph library
import rustworkx as rx
from rustworkx.visualization import mpl_draw


# To run on hardware, select the backend with the fewest number of jobs in the queue
service = QiskitRuntimeService(channel="ibm_quantum",token="2e7c6786896965a581a4b8bf14ad4e4e0d68d0998eead1ffa73e6afbca766bee9e2f623010856aaede78b5def50a786015329192dae5f468292e5ad12a9c3ae0")
backend = service.least_busy(min_num_qubits=40,operational=True, simulator=False)
backend.name

'ibm_brisbane'

In [5]:
op,offset = qp.to_ising()
print("offset: {}".format(offset))
print("operator:")
print(op)

offset: 629.7026127910186
operator:
SparsePauliOp(['IIIIIIIIIIIIIIIIIIIZ', 'IIIIIIIIIIIIIIIIIIZI', 'IIIIIIIIIIIIIIIIIZII', 'IIIIIIIIIIIIIIIIZIII', 'IIIIIIIIIIIIIIIZIIII', 'IIIIIIIIIIIIIIZIIIII', 'IIIIIIIIIIIIIZIIIIII', 'IIIIIIIIIIIIZIIIIIII', 'IIIIIIIIIIIZIIIIIIII', 'IIIIIIIIIIZIIIIIIIII', 'IIIIIIIIIZIIIIIIIIII', 'IIIIIIIIZIIIIIIIIIII', 'IIIIIIIZIIIIIIIIIIII', 'IIIIIIZIIIIIIIIIIIII', 'IIIIIZIIIIIIIIIIIIII', 'IIIIZIIIIIIIIIIIIIII', 'IIIZIIIIIIIIIIIIIIII', 'IIZIIIIIIIIIIIIIIIII', 'IZIIIIIIIIIIIIIIIIII', 'ZIIIIIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIZZ', 'IIIIIIIIIIIIIIIIIZIZ', 'IIIIIIIIIIIIIIIIZIIZ', 'IIIIIIIIIIIIIIIZIIIZ', 'IIIIIIIIIIIIIIZIIIIZ', 'IIIIIIIIIIIIIZIIIIIZ', 'IIIIIIIIIIIIZIIIIIIZ', 'IIIIIIIIIIIZIIIIIIIZ', 'IIIIIIIIIIZIIIIIIIIZ', 'IIIIIIIIIZIIIIIIIIIZ', 'IIIIIIIIZIIIIIIIIIIZ', 'IIIIIIIZIIIIIIIIIIIZ', 'IIIIIIZIIIIIIIIIIIIZ', 'IIIIIZIIIIIIIIIIIIIZ', 'IIIIIIIIIIIIIIIIIZZI', 'IIIIIIIIIIIIIIIIZIZI', 'IIIIIIIIIIIIIIIZIIZI', 'IIIIIIIIIIIIIIZIIIZI', 'IIIIIIIIIIIIIZIIIIZI', 'IIIIIIIIIIII

In [6]:
from qiskit.circuit.library import QAOAAnsatz
ansatz = QAOAAnsatz(op, reps=2)

ansatz.decompose(reps=3).draw(output="mpl", style="iqp")

In [8]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)

ansatz_isa = pm.run(ansatz)

In [9]:
hamiltonian_isa = op.apply_layout(ansatz_isa.layout)
hamiltonian_isa

SparsePauliOp(['IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII

In [10]:
def cost_func(params, ansatz, hamiltonian, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (EstimatorV2): Estimator primitive instance

    Returns:
        float: Energy estimate
    """
    pub = (ansatz, [hamiltonian], [params])
    result = estimator.run(pubs=[pub]).result()
    cost = result[0].data.evs[0]

    return cost

In [11]:
session = Session(backend=backend)

# Configure estimator
estimator = Estimator(session=session)
estimator.options.default_shots = 10_000
estimator.options.dynamical_decoupling.enable = True

# Configure sampler
sampler = Sampler(session=session)
sampler.options.default_shots = 10_000
sampler.options.dynamical_decoupling.enable = True

In [12]:
x0 = 2 * np.pi * np.random.rand(ansatz_isa.num_parameters)

In [ ]:
res = minimize(cost_func, x0, args=(ansatz_isa, hamiltonian_isa, estimator), method="COBYLA")
res